In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time

In [42]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

In [23]:
base_url = "https://lifehacker.ru/topics/technology/?page={}"

article_links = []

In [65]:
for page in range(1, 11):
    url = f"{base_url}?page={page}"
    print(f"Загружается страница: {url}")
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        links = soup.find_all("a", class_="lh-small-article-card__link")

        for a_tag in links:
            href = a_tag.get("href")
            if href and href.startswith("/"):
                article_links.append("https://lifehacker.ru" + href)

        time.sleep(1)
    else:
        print(f"Ошибка при загрузке страницы {page}: {response.status_code}")

article_links = list(set(article_links))
print(f"\nНайдено уникальных ссылок: {len(article_links)}\n")

Загружается страница: https://lifehacker.ru/topics/technology/?page=1
Загружается страница: https://lifehacker.ru/topics/technology/?page=2
Загружается страница: https://lifehacker.ru/topics/technology/?page=3
Загружается страница: https://lifehacker.ru/topics/technology/?page=4
Загружается страница: https://lifehacker.ru/topics/technology/?page=5
Загружается страница: https://lifehacker.ru/topics/technology/?page=6
Загружается страница: https://lifehacker.ru/topics/technology/?page=7
Загружается страница: https://lifehacker.ru/topics/technology/?page=8
Загружается страница: https://lifehacker.ru/topics/technology/?page=9
Загружается страница: https://lifehacker.ru/topics/technology/?page=10

Найдено уникальных ссылок: 299



In [77]:
articles_data = []

for link in tqdm(article_links, desc="Парсинг статей"):
    try:
        response = requests.get(link, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")

            title_tag = soup.find("h1")
            title = title_tag.get_text(strip=True) if title_tag else "Нет заголовка"

            p_tags = soup.find_all("p", attrs={"data-v-98c39174": True})
            if not p_tags:
                p_tags = soup.find_all("p")
            text = " ".join([p.get_text(strip=True) for p in p_tags]) if p_tags else "Нет текста"

            articles_data.append({
                "title": title,
                "text": text
            })

            time.sleep(1)
    except Exception as e:
        print(f"Ошибка при обработке {link}: {e}")

Парсинг статей: 100%|██████████| 299/299 [12:25<00:00,  2.49s/it]


In [78]:
pd.set_option('display.max_colwidth', 400)

data = pd.DataFrame(articles_data)
data.head()

,title,text
0,"Обзор Le Chat — бесплатного французского чат-бота с редактором текстов, как в ChatGPT","Автор Лайфхакера Похоже, мы переживаем ренессанс чат-ботов: аналоги ChatGPT появляются один за другим. Причём действительно достойные, а не как раньше: «Есть диалоговое окно? Умеет что-то писать? Срочно зачисляем в убийцы ChatGPT!» Сейчас с детищем OpenAI конкурируют Gemini, Claude,Copilot,DeepSeek,Qwen, а также модели без официальных интерфейсов вроде Mixtral. Долгое время на рынке доминирова..."
1,Garmin представила часы Descent G2 для дайвинга — это конкурент Apple Watch Ultra 2,"Garminпредставиламногофункциональные часы Descent G2, разработанные для занятий спортом и дайвингом. В компании называют их «компьютером для погружений». Новинка способна выдерживать давление воды на глубине до 100 метров. Устройство оснащено 1,2‑дюймовым AMOLED-дисплеем из сапфирового стекла, который обеспечивает четкую видимость данных даже под водой. Корпус и безель изготовлены из прочного ..."
2,Как открыть скрытые папки Windows,"По умолчанию Windows прячет некоторые папки, чтобы защитить важные системные файлы и не перегружать интерфейс. Перед тем как открыть скрытые папки, надо учесть, что они бывают двух типов: системные и обычные. В системных папках, как понятно, содержатся файлы, необходимые для работы ОС. Windows скрывает их, чтобы пользователь случайно не удалил что-то важное. В обычных скрытых папках содержатся..."
3,"8 крутых смартфонов, которые сильно подешевели в 2025 году","Рынок смартфонов переживает лучшие для покупателей (но не лучшие для их производителей) времена: современные модели стремительно дешевеют уже через несколько месяцев после релиза, при этом их возможности остаются актуальными, даже когда выпускают следующее поколение. В этом прелесть плато технологий, на которое мы вышли: покупать самое новое уже не так важно, да и «старое»поддерживают дольше. ..."
4,Что такое VoLTE: 3 факта о технологии передачи голоса,"VoLTE — технология, которая проводит голосовые звонки через сеть LTE (4G). И она постепенно замещает привычный формат передачи сигнала по 2G или 3G. Причина — ускорение связи. VoLTEиспользуеттехнологию коммутации пакетов: для неё голосовая связь — набор цифровых данных. В звонках по 2G/3G-сетям применяется коммутация каналов. Чтобы передать сигнал, нужен отдельный путь, переключение на который..."
